# Introducing tf.estimator

**Learning Objectives**
  - Understand where the `tf.estimator` module sits in the hierarchy of Tensorflow APIs
  - Understand the workflow of creating a `tf.estimator` model
    1. Create Feature Columns
    2. Create Input Functions
    3. Create Estimator
    4. Train/Evaluate/Predict
  - Understand how to swap in/out different types of Estimators
  
## Introduction

Tensorflow is a hierarchical framework. The further down the hierarchy you go, the more flexibility you have, but that more code you have to write. Generally one starts at the highest level of abstraction. Then if you need additional flexibility drop down one layer.

<img src='assets/TFHierarchy.png' width='50%'>
<sup>(image: https://www.tensorflow.org/guide/premade_estimators)</sup>

In this notebook we will be operating at the highest level of Tensorflow abstraction, using the Estimator API to predict taxifare prices on the sampled dataset we created previously.

In [1]:
# Ensure that we have Tensorflow 1.13.1 installed.
!pip3 freeze | grep tensorflow==1.13.1 || pip3 install tensorflow==1.13.1

    100% |████████████████████████████████| 92.5MB 13kB/s 
    100% |████████████████████████████████| 2.2MB 689kB/s 
    100% |████████████████████████████████| 368kB 3.6MB/s 
    100% |████████████████████████████████| 51kB 12.3MB/s 
    100% |████████████████████████████████| 1.2MB 1.2MB/s 
    100% |████████████████████████████████| 3.2MB 478kB/s 
    100% |████████████████████████████████| 102kB 7.5MB/s 
    100% |████████████████████████████████| 51kB 10.6MB/s 
    100% |████████████████████████████████| 20.2MB 70kB/s 
    100% |████████████████████████████████| 583kB 2.5MB/s 
    100% |████████████████████████████████| 337kB 4.2MB/s 
    100% |████████████████████████████████| 92kB 10.7MB/s 
    100% |████████████████████████████████| 2.8MB 519kB/s 
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Running setup.py bdist_wheel for absl-py ... done
  Stored in direc

In [2]:
import tensorflow as tf
import pandas as pd
import shutil

print(tf.__version__)

1.14.0


## Load raw data 

First, let's download the raw .csv data. These are the same files created in the `create_datasets.ipynb` notebook

In [3]:
!gsutil cp gs://cloud-training-demos/taxifare/small/*.csv .
!ls -l *.csv

Copying gs://cloud-training-demos/taxifare/small/taxi-test.csv...
Copying gs://cloud-training-demos/taxifare/small/taxi-train.csv...
Copying gs://cloud-training-demos/taxifare/small/taxi-valid.csv...
/ [3 files][ 10.9 MiB/ 10.9 MiB]                                                
Operation completed over 3 objects/10.9 MiB.                                     
-rw-r--r-- 1 jupyter jupyter 1799474 Jul 29 14:28 taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 7986353 Jul 29 14:28 taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 1673742 Jul 29 14:28 taxi-valid.csv


Because the files are small we can load them into in-memory Pandas dataframes.

In [4]:
df_train = pd.read_csv(filepath_or_buffer = "./taxi-train.csv")
df_valid = pd.read_csv(filepath_or_buffer = "./taxi-valid.csv")
df_test = pd.read_csv(filepath_or_buffer = "./taxi-test.csv")

CSV_COLUMN_NAMES = list(df_train)
print(CSV_COLUMN_NAMES)

FEATURE_NAMES = CSV_COLUMN_NAMES[1:] # all but first column
LABEL_NAME = CSV_COLUMN_NAMES[0] # first column

['fare_amount', 'dayofweek', 'hourofday', 'pickuplon', 'pickuplat', 'dropofflon', 'dropofflat']


## Create feature columns

Feature columns make it easy to perform common type of feature engineering on your raw data. For example you can one-hot encode categorical data, create feature crosses, embeddings and more. We'll cover these later in the course, but if you want to a sneak peak browse the official TensorFlow [feature columns guide](https://www.tensorflow.org/guide/feature_columns).

In our case we won't do any feature engineering. However we still need to create a list of feature columns because the Estimator we will use requires one. To specify the numeric values should be passed on without modification we use `tf.feature_column.numeric_column()`

We use a [python list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) to create the list of feature columns, which is just an elegant alternative to a for loop.

In [5]:
feature_columns = [tf.feature_column.numeric_column(key = k) for k in FEATURE_NAMES]
feature_columns

[NumericColumn(key='dayofweek', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='hourofday', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Define input function

Now that your estimator knows what type of data to expect and how to intepret it, you need to actually pass the data to it! This is the job of the input function.

The input function returns a new batch of (features, label) tuples each time it is called by the Estimator.

- features: A python dictionary. Each key is a feature column name and its value is the tensor containing the data for that feature
- label: A Tensor containing the labels

So how do we get from our current Pandas dataframes to (features, label) tuples that return one batch at a time?

The `tf.data` module contains a collection of classes that allows you to easily load data, manipulate it, and pipe it into your model. https://www.tensorflow.org/guide/datasets_for_estimators

In [6]:
def train_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = (dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    # Note:
    # If we returned now, the Dataset would iterate over the data once  
    # in a fixed order, and only produce a single element at a time.
    
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count = None).batch(batch_size = batch_size)
   
    return dataset

def eval_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = (dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
   
    return dataset

def predict_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = dict(df[FEATURE_NAMES])) # no label

    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
   
    return dataset

## Choose Estimator

Tensorflow has several premade estimators for you to choose from:

- LinearClassifier/Regressor
- BoostedTreesClassifier/Regressor
- DNNClassifier/Regressor
- DNNLinearCombinedClassifier/Regressor

If none of these meet your needs you can implement a custom estimator using `tf.Keras`. We'll cover that later in the course.

For now we will use the premade LinearRegressor. To instantiate an estimator simply pass it what feature columns to expect and specify an directory for it to output checkpoint files to.

In [7]:
OUTDIR = "taxi_trained"

model = tf.estimator.LinearRegressor(
    feature_columns = feature_columns,
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed = 1) # for reproducibility
)

## Train

Simply invoke the estimator's `train()` function. Specify the `input_fn` which tells it how to load in data, and specify the number of steps to train for.

By default estimators check the output directory for checkpoint files before beginning training, so it can pickup where it last left off. To prevent this we'll delete the output directory before starting training each time.

In [8]:
%%time
tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(path = OUTDIR, ignore_errors = True) # start fresh each time

model.train(
    input_fn = lambda: train_input_fn(df = df_train), 
    steps = 500)

W0729 14:28:17.316154 139729986717440 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0729 14:28:17.377522 139729986717440 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0729 14:28:17.395714 139729986717440 estimator.py:1145] Calling model_fn.
W0729 14:28:17.972416 139729986717440 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/pyth

CPU times: user 8.78 s, sys: 2.82 s, total: 11.6 s
Wall time: 6.04 s


## Evaluate

Estimators similarly have an `evaluate()` function. In this case we don't need to specify the number of steps to train because we didn't tell our input function to repeat the data. Once the input function reaches the end of the data evaluation will end. 

Loss is reported as MSE by default so we take the square root before printing.

In [9]:
def print_rmse(model, df):
    metrics = model.evaluate(input_fn = lambda: eval_input_fn(df))
    print("RMSE on dataset = {}".format(metrics["average_loss"]**.5))
print_rmse(model = model, df = df_valid)

I0729 14:28:23.392342 139729986717440 estimator.py:1145] Calling model_fn.
I0729 14:28:24.022541 139729986717440 estimator.py:1147] Done calling model_fn.
I0729 14:28:24.046400 139729986717440 evaluation.py:255] Starting evaluation at 2019-07-29T14:28:24Z
I0729 14:28:24.166563 139729986717440 monitored_session.py:240] Graph was finalized.
W0729 14:28:24.168464 139729986717440 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0729 14:28:24.170772 139729986717440 saver.py:1280] Restoring parameters from taxi_trained/model.ckpt-500
I0729 14:28:24.235679 139729986717440 session_manager.py:500] Running local_init_op.
I0729 14:28:24.267910 139729986717440 session_manager.py:502] Done running local_init_op.
I0729 14:28:25.53

RMSE on dataset = 9.459564487360257


RMSE of 9.45 is  worse than our rules based benchmark (RMSE of $7.70). However given that we haven't done any feature engineering or hyperparameter tuning, and we're training on a small dataset using a simple linear model, we shouldn't yet expect good performance. 

The goal at this point is to demonstrate the mechanics of the Estimator API. In subsequent notebooks we'll improve on the model.

## Predict

To make predictions with our trained model, we call the `predict` method of our estimator, passing to it the input function to read in the data. Here we make predictions for the first 10 elements of the `df_test` dataframe. 

In [10]:
predictions = model.predict(input_fn = lambda: predict_input_fn(df = df_test[:10]))
for items in predictions:
    print(items)

I0729 14:28:25.669340 139729986717440 estimator.py:1145] Calling model_fn.
I0729 14:28:25.965878 139729986717440 estimator.py:1147] Done calling model_fn.
I0729 14:28:26.271385 139729986717440 monitored_session.py:240] Graph was finalized.
I0729 14:28:26.278640 139729986717440 saver.py:1280] Restoring parameters from taxi_trained/model.ckpt-500
I0729 14:28:26.336723 139729986717440 session_manager.py:500] Running local_init_op.
I0729 14:28:26.346451 139729986717440 session_manager.py:502] Done running local_init_op.


{'predictions': array([9.537243], dtype=float32)}
{'predictions': array([9.708608], dtype=float32)}
{'predictions': array([9.596281], dtype=float32)}
{'predictions': array([9.537495], dtype=float32)}
{'predictions': array([9.537642], dtype=float32)}
{'predictions': array([9.879642], dtype=float32)}
{'predictions': array([9.879233], dtype=float32)}
{'predictions': array([9.70439], dtype=float32)}
{'predictions': array([9.535185], dtype=float32)}
{'predictions': array([9.877537], dtype=float32)}


Further evidence of the primitiveness of our model, it predicts similar amounts for every trip!

## Change Estimator type

One of the payoffs for using the Estimator API is we can swap in a different model type with just a few lines of code. Let's try a DNN. Note how now we need to specify the number of neurons in each hidden layer.

In [11]:
%%time

tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(path = OUTDIR, ignore_errors = True) 
model = tf.estimator.DNNRegressor(
    hidden_units = [10,10], # specify neural architecture
    feature_columns = feature_columns, 
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed = 1)
)
model.train(
    input_fn = lambda: train_input_fn(df = df_train), 
    steps = 500)
print_rmse(model = model, df = df_valid)

I0729 14:28:26.529272 139729986717440 estimator.py:209] Using config: {'_num_ps_replicas': 0, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_distribute': None, '_task_type': 'worker', '_tf_random_seed': 1, '_master': '', '_is_chief': True, '_save_summary_steps': 100, '_task_id': 0, '_model_dir': 'taxi_trained', '_experimental_max_worker_delay_secs': None, '_service': None, '_eval_distribute': None, '_device_fn': None, '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_train_distribute': None, '_save_checkpoints_secs': 600, '_protocol': None, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1560208c50>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}
I0729 14:28:26.580809 139729986717440 estimator.py:1145] Calling model_fn.
W0729 14:28:26.585566 1397

RMSE on dataset = 9.2532314120496
CPU times: user 10.6 s, sys: 2.67 s, total: 13.3 s
Wall time: 7.33 s


Our performance is only slightly better at 9.25, and still far worse than our rules based model.  This illustrates an important tenant of machine learning: A more complex model can't outrun bad data. 

Currently since we're not doing any feature engineering our input data has very little signal to learn from, so using a DNN doesn't help much.

## Results summary

We can summarize our results in a table here.
   
|Model | RMSE on validation set|
|------|-----------------|
|Rules Based Benchmark| 7.76|
|Linear Model| 9.45|
|DNN Model|9.26

## Challenge exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License